# Bag of Words Sentiment Analyse von McDonalds Reviews und IMBD Daten

## Zielsetzung 
In diesem Notebook erweitern wir die McDonald's-Daten mit IMDB-Daten, um die Auswirkungen auf die Genauigkeit zu erforschen. Dafür teilen wir die Daten in positive und negative Reviews auf, was zu nur zwei Labels führt. Die Baseline basierend auf dem gesunden Menschenverstand liegt daher bei einer Genauigkeit von 50%.
 Unser Ziel ist es herauszufinden, ob wir durch Hinzufügen immer mehr Daten eine allgemeine Semantikanalyse erreichen können, die auf jede beliebige Review angewendet werden kann.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers


2023-07-09 12:31:59.030006: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 12:31:59.057791: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 12:31:59.470662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Datenvorbereitung inkl. Behandlung bekannter Probleme

### Laden der Daten

In [ ]:
%pip install kaggle

In [3]:
import os

# Download McDonalds dataset
if not os.path.exists('mcdonalds-store-reviews.zip'):
    print("Downloading McDonalds dataset...")
    !kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
if os.path.exists('mcdonalds-store-reviews.zip'):
    print("Unzipping McDonalds dataset...")
    !unzip -n mcdonalds-store-reviews.zip

# Download IMDB dataset
if not os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Downloading IMDB dataset...")
    !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
if os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Unzipping IMDB dataset...")
    !unzip -n imdb-dataset-of-50k-movie-reviews.zip

Unzipping McDonalds dataset...
/usr/bin/zsh: /home/x/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Archive:  mcdonalds-store-reviews.zip
Unzipping IMDB dataset...
/usr/bin/zsh: /home/x/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Archive:  imdb-dataset-of-50k-movie-reviews.zip


In [4]:
df_mc = pd.read_csv('McDonald_s_Reviews.csv', encoding="latin-1")
df_imdb = pd.read_csv('IMDB Dataset.csv')

In [5]:
df_mc.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [6]:
df_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Bereinigung der Daten

In [13]:
from utils.text_utils import clean_text
df_mc['review'] = df_mc['review'].apply(clean_text)
df_imdb['sentiment'] = df_imdb['review'].apply(clean_text)

df_mc = df_mc[['review', 'rating']]
df_imdb = df_imdb[['review', 'sentiment']]

KeyboardInterrupt: 

In [11]:
df_mc.head()

,review,rating
0,look like someone spit food normal transaction...,1 star
1,itd mcdonalds far food atmosphere go staff mak...,4 stars
2,made mobile order got speaker checked line mov...,1 star
3,mc crispy chicken sandwich customer service qu...,5 stars
4,repeat order times drive thru still manage mes...,1 star


In [12]:
df_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Löschen der 3 Sterne Bewertungen beim McDonalds Dataset

In [ ]:
df_mc = df_mc[df_mc['rating'] != '3 stars']

In [ ]:
print(df_mc.columns)
print(df_imdb.columns)

### Konvertierung der train data zu numpy arrays

In [ ]:
data_mc = df_mc['review'].to_numpy()
data_imdb = df_imdb['review'].to_numpy()


### Konvertierung der Datasets von string labels zu Nummern

In [ ]:
rating_mapping_imdb = {
    'positive': 1,
    'negative': 0,
}

label_imdb = df_imdb['sentiment'].map(rating_mapping_imdb).to_numpy()


In [ ]:
rating_mapping_mc = {
    '1 star': 0,
    '2 stars': 0,
    '4 stars': 1,
    '5 stars': 1
}

label_mc = df_mc['rating'].map(rating_mapping_mc).to_numpy()

### Daten zusammenführen

In [ ]:
data = np.append(data_imdb, data_mc)
label = np.append(label_imdb,label_mc)
print(data.shape)
print(label.shape)

### Aufteilung in Trainings, Validierungs und Testdaten

In [ ]:
train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2, random_state=42)

print(train_data.shape)
print(test_data.shape)
print(train_label.shape)
print(test_label.shape)

#### McDonalds Train/Test split

Um die Veränderung des Ergbnisses, durch das Einführen von Testsdaten vergleichen zu können, werden wir später auch noch ein Model nur mit den McD Daten trainieren

In [ ]:
train_data_mc, test_data_mc, train_label_mc, test_label_mc = train_test_split(data_mc, label_mc, test_size=0.2, random_state=42)

print(train_data_mc.shape)
print(test_data_mc.shape)
print(train_label_mc.shape)
print(test_label_mc.shape)

#### IMDB Train/Test split

Um die Veränderung des Ergbnisses, durch das Einführen von Testsdaten vergleichen zu können, werden wir später auch noch ein Model nur mit den McD Daten trainieren

In [ ]:
train_data_imdb, test_data_imdb, train_label_imdb, test_label_imdb = train_test_split(data_imdb, label_imdb, test_size=0.2, random_state=42)

print(train_data_imdb.shape)
print(test_data_imdb.shape)
print(train_label_imdb.shape)
print(test_label_imdb.shape)

### Text zu Dictonary Repräsentationen umbauen
Erklärung Menge an Tokens
Erklärung ngrams 
Erklärung multi_hot und warum

In [ ]:
text_vectorization = TextVectorization(
 max_tokens=20000,
ngrams=2,
 output_mode="multi_hot",
 standardize=None,
)

In [ ]:
text_vectorization.adapt(train_data) 
text_vectorization.adapt(test_data) 

multi_hot_train_data = text_vectorization(train_data)
multi_hot_test_data = text_vectorization(test_data)

#### Text Vectorization für McD

In [ ]:
multi_hot_train_data_mc = text_vectorization(train_data_mc)
multi_hot_test_data_mc = text_vectorization(test_data_mc)

#### Text Vectorization für IMDB

In [ ]:
multi_hot_train_data_imdb = text_vectorization(train_data_imdb)
multi_hot_test_data_imdb = text_vectorization(test_data_imdb)

## Erstellung eines entsprechenden künstlichen neuronalen Models sowie geeigente Benchmarks

In [ ]:
def get_model(max_tokens=20000, hidden_dim=64):
 inputs = keras.Input(shape=(max_tokens,))
 x = layers.Dense(hidden_dim, activation="relu")(inputs)
 x = layers.Dense(32, activation="relu")(x)
 x = layers.Dropout(0.5)(x)
 x = layers.Dense(16, activation="relu")(x)
 outputs = layers.Dense(1, activation="sigmoid")(x)
 model = keras.Model(inputs, outputs)
 model.compile(optimizer="rmsprop",
 loss="binary_crossentropy",
 metrics=["accuracy"], 
 )
 return model

model = get_model()
model.summary()


In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]

In [ ]:
history = model.fit(multi_hot_train_data,train_label,
validation_split=0.2,
 epochs=10, 
 callbacks= callbacks
 )

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Evaluierung auf alle Daten


In [ ]:
model.evaluate(multi_hot_test_data, test_label)

### Evalurierung nur auf die McDonalds Daten

In [ ]:
model.evaluate(multi_hot_test_data_mc, test_label_mc)

### Evalurierung nur auf die IMDB Daten

In [ ]:
model.evaluate(multi_hot_test_data_imdb, test_label_imdb)

## Vergleich nur McDonalds Daten

In [ ]:
model_mc = get_model()
model_mc.fit(multi_hot_train_data_mc,train_label_mc,
validation_split=0.2,
 epochs=10, 
 callbacks= callbacks
 )

### McDonalds Model auf McDonalds Model Testen

In [ ]:
model_mc.evaluate(multi_hot_test_data_mc, test_label_mc)

### McDonalds Model auf IMDB Daten testen

In [ ]:
model_mc.evaluate(multi_hot_test_data_imdb, test_label_imdb)

### McDonalds Model auf allgemeine Daten testen

In [ ]:
model_mc.evaluate(multi_hot_test_data, test_label)

## Vergleich IMDB Model

In [ ]:

model_imdb = get_model()
model_imdb.fit(multi_hot_train_data_imdb,train_label_imdb,
validation_split=0.2,
 epochs=10, 
 callbacks= callbacks
 )

### IMDB Model auf IMDB Model Testen

In [ ]:
model_imdb.evaluate(multi_hot_test_data_imdb, test_label_imdb)

### IMDB Model auf McDonalds Daten Testen

In [ ]:
model_imdb.evaluate(multi_hot_test_data_mc, test_label_mc)

### IMDB Model auf allgemeines Datenset

In [ ]:
model_imdb.evaluate(multi_hot_test_data, test_label)

## Fazit


Mehr Daten ermöglichen die Lösung allgemeinerer Probleme, führen jedoch zu einer verringerten Genauigkeit bei spezifischeren Modellen. Ein Problem in unserem Modell besteht darin, dass wir nicht gleich viele Trainingsdaten von IMDB und McDonald's haben, wodurch das Modell stärker auf Movie Reviews gewichtet wird.

Das allgemeine Modell erzielt eine Genauigkeit von 90% auf den allgemeinen Daten, aber bei der Evaluierung nur auf die McDonald's- oder IMDB-Reviews schafft es knapp oder gar nicht, die Common-Sense-Baseline zu übertreffen. Dennoch sind die Ergebnisse hier etwas besser als bei der Verwendung des IMDB-Modells auf den McDonald's-Daten oder des McDonald's-Modells auf den IMDB-Daten. Daher wäre anzunehmen, dass die Genauigkeit steigen würde, wenn wir mehr Reviews hinzufügen (z. B. Amazon-Reviews). Dadurch könnte langfristig eine semantische Analyse für alle Arten von Reviews erstellt werden.